# Code 1

In [56]:
def ConsomCoulee(times, PFC_values, ListeMasse_grappe, ListeQuantite_mottes, nb_moules_heure, t_serie):
    PFC = PFC_values[-1]
    tmin = times[-1]
    t = tmin
    for masse_grappe_i, Quantite_mottes_i in zip(ListeMasse_grappe, ListeQuantite_mottes) :
        for i in range(Quantite_mottes_i):
            cadences_i = nb_moules_heure/60 * masse_grappe_i  # consomation de fonte par minute (kg/min)
            PFC = PFC - cadences_i  # en kg
            t+=1
            times.append(t)
            PFC_values.append(PFC)
        t+=t_serie
        times.append(t)
        PFC_values.append(PFC)
    tmax = t
    return times, PFC_values, tmin, tmax

import plotly.graph_objects as go

def Plot_consom(times, PFC_values, tmin, tmax, PFCmin, PFCmax):

    # Create Plotly figure
    fig = go.Figure()

    # Add PFC line
    fig.add_trace(go.Scatter(x=times, y=PFC_values, mode='lines', name='PFC over time', line=dict(color='royalblue')))

    # Add horizontal lines for PFCmin and PFCmax
    fig.add_shape(type="line", x0=tmin, y0=PFCmin, x1=tmax, y1=PFCmin, line=dict(color="firebrick", width=2, dash="dash"), name="PFCmin")
    
    fig.add_shape(type="line", x0=tmin, y0=PFCmax, x1=tmax, y1=PFCmax, line=dict(color="green", width=2, dash="dash"), name="PFCmax")


    # Customize layout
    fig.update_layout(title='Consommation de fonte au cours du temps', xaxis_title='Time (minutes)', yaxis_title='PFC (kg)', template='plotly',
        showlegend=False,annotations=[
            dict(x=tmax,y=PFCmin,xref="x",yref="y",text="PFCmin",showarrow=True,arrowhead=2,ax=20,ay=-30),
            dict(x=tmax,y=PFCmax,xref="x",yref="y",text="PFCmax",showarrow=True,arrowhead=2,ax=20,ay=-30)])

    # Show the plot
    fig.show()


In [57]:
# Les valeurs d'entrée Input 

nb_moules_heure = 160 
t_serie = 5 # 5 min

PFCmin = 2500 # en kg
PFCmax = 4000 # en kg

times = [0] # en min
PFC_values = [4000] # en kg
ListeMasse_grappe = [35, 10, 20, 35] # en kg
ListeQuantite_mottes = [50, 50, 50, 50]

times, PFC_values, tmin, tmax = ConsomCoulee(times, PFC_values, ListeMasse_grappe, ListeQuantite_mottes, nb_moules_heure, t_serie)

In [58]:
Plot_consom(times, PFC_values, tmin, tmax, PFCmin, PFCmax)

# Code 2

In [36]:
import plotly.graph_objects as go

def Plot_consom(times, PFC_values, tmin, tmax, PFCmin, PFCmax):

    # Create Plotly figure
    fig = go.Figure()

    # Add PFC line
    fig.add_trace(go.Scatter(x=times, y=PFC_values, mode='lines', name='PFC over time', line=dict(color='royalblue')))

    # Add horizontal lines for PFCmin and PFCmax
    fig.add_shape(type="line", x0=tmin, y0=PFCmin, x1=tmax, y1=PFCmin, line=dict(color="firebrick", width=2, dash="dash"), name="PFCmin")
    
    fig.add_shape(type="line", x0=tmin, y0=PFCmax, x1=tmax, y1=PFCmax, line=dict(color="green", width=2, dash="dash"), name="PFCmax")


    # Customize layout
    fig.update_layout(title='Consommation de fonte au cours du temps', xaxis_title='Time (minutes)', yaxis_title='PFC (kg)', template='plotly',
        showlegend=False,annotations=[
            dict(x=tmax,y=PFCmin,xref="x",yref="y",text="PFCmin",showarrow=True,arrowhead=2,ax=20,ay=-30),
            dict(x=tmax,y=PFCmax,xref="x",yref="y",text="PFCmax",showarrow=True,arrowhead=2,ax=20,ay=-30)])

    # Show the plot
    fig.show()

def Plot_Prod(times, PFC_values):

    # Create Plotly figure
    fig = go.Figure()

    # Add PFC line
    fig.add_trace(go.Scatter(x=times, y=PFC_values, mode='lines', name='PFC over time', line=dict(color='royalblue')))

    # Customize layout
    fig.update_layout(title='Production de fonte au cours du temps', xaxis_title='Time (minutes)', yaxis_title='PFC (kg)', template='plotly',
        showlegend=False)

    # Show the plot
    fig.show()

In [48]:
def ProdConsom(ListeMasse_grappe, ListeQuantite_mottes, nb_moules_heure, t_serie, Temps_fusion, Temps_chargement):

    numfours = 0
    Fonte_produit = 5000 # Fontes produits toute les 45 min en kg
    PFC_Prodvalues = [0] # en kg
    PFC_Prod =  PFC_Prodvalues[-1]


    PFC_Consomvalues = [4000] # en kg
    PFC_Consom = PFC_Consomvalues[-1]
    times = [0] # en min
    tmin = times[-1]
    t = tmin

    for masse_grappe_i, Quantite_mottes_i in zip(ListeMasse_grappe, ListeQuantite_mottes) :
        for i in range(Quantite_mottes_i):
            cadences_i = nb_moules_heure/60 * masse_grappe_i  # consomation de fonte par minute (kg/min)
            PFC_Consom = PFC_Consom - cadences_i  # en kg
            t+=1
            times.append(t)
            PFC_Consomvalues.append(PFC_Consom)

            #  Partie Producion de la fonte dans le temps
            if t <= Temps_chargement:
                # Avant ou pendant Temps_chargement
                PFC_Prodvalues.append(PFC_Prod)
            else:
                # Après Temps_chargement
                if (t - Temps_chargement) % Temps_fusion == 0:
                    numfours += 1
                    # print(t,numfours)
                    PFC_Prod += Fonte_produit
                PFC_Prodvalues.append(PFC_Prod)
            
        t+=t_serie
        #  Partie Producion de la fonte dans le temps
        if t <= Temps_chargement:
            # Avant ou pendant Temps_chargement
            PFC_Prodvalues.append(PFC_Prod)
        else:
            # Après Temps_chargement
            if (t - Temps_chargement) % Temps_fusion == 0:
                numfours += 1
                # print(t,numfours)
                PFC_Prod += Fonte_produit
            PFC_Prodvalues.append(PFC_Prod)

        times.append(t)
        PFC_Consomvalues.append(PFC_Consom)
    tmax = t
    return times, PFC_Consomvalues, PFC_Prodvalues, tmin, tmax

In [49]:
nb_fours = 6 # 8-9 fours pour HWS, 6 fours pour DiSA
Temps_fusion = 45 # en min
Temps_chargement = 30 # 30 - 45  min

# Les valeurs d'entrée Input 

nb_moules_heure = 160 
t_serie = 5 # 5 min

PFCmin = 2500 # en kg
PFCmax = 4000 # en kg

times = [0] # en min
PFC_values = [4000] # en kg
ListeMasse_grappe = [35, 10, 20, 35] # en kg
ListeQuantite_mottes = [50, 50, 50, 50]

times, PFC_Consomvalues, PFC_Prodvalues, tmin, tmax = ProdConsom(ListeMasse_grappe, ListeQuantite_mottes, nb_moules_heure, t_serie, Temps_fusion, Temps_chargement)

In [50]:
Plot_consom(times, PFC_Consomvalues, tmin, tmax, PFCmin, PFCmax)

In [51]:
Plot_Prod(times, PFC_Prodvalues)

# Code 3

In [133]:
import plotly.graph_objects as go

def Plot_consom(times, PFC_values, tmin, tmax, PFCmin, PFCmax):

    # Create Plotly figure
    fig = go.Figure()

    # Add PFC line
    fig.add_trace(go.Scatter(x=times, y=PFC_values, mode='lines', name='PFC over time', line=dict(color='royalblue')))

    # Add horizontal lines for PFCmin and PFCmax
    fig.add_shape(type="line", x0=tmin, y0=PFCmin, x1=tmax, y1=PFCmin, line=dict(color="firebrick", width=2, dash="dash"), name="PFCmin")
    
    fig.add_shape(type="line", x0=tmin, y0=PFCmax, x1=tmax, y1=PFCmax, line=dict(color="green", width=2, dash="dash"), name="PFCmax")


    # Customize layout
    fig.update_layout(title='Consommation de fonte au cours du temps', xaxis_title='Time (minutes)', yaxis_title='PFC (kg)', template='plotly',
        showlegend=False,annotations=[
            dict(x=tmax,y=PFCmin,xref="x",yref="y",text="PFCmin",showarrow=True,arrowhead=2,ax=20,ay=-30),
            dict(x=tmax,y=PFCmax,xref="x",yref="y",text="PFCmax",showarrow=True,arrowhead=2,ax=20,ay=-30)])

    # Show the plot
    fig.show()

def Plot_Prod(times, PFC_values):

    # Create Plotly figure
    fig = go.Figure()

    # Add PFC line
    fig.add_trace(go.Scatter(x=times, y=PFC_values, mode='lines', name='PFC over time', line=dict(color='royalblue')))

    # Customize layout
    fig.update_layout(title='Production de fonte au cours du temps', xaxis_title='Time (minutes)', yaxis_title='PFC (kg)', template='plotly',
        showlegend=False)

    # Show the plot
    fig.show()

In [228]:
def calculate_cadence_per_minute(nb_moules_heure, masse_grappe):
    """
    Calcule la cadence de consommation de fonte par minute.
    """
    return nb_moules_heure / 60 * masse_grappe

def update_production(t, Temps_chargement, Temps_fusion, numfours, Fonte_produit, PFC_Prod, poche = 0):
    """
    Met à jour la production de fonte en fonction du temps écoulé.
    """
    if t > Temps_chargement and (t - Temps_chargement) % Temps_fusion == 0:
        numfours += 1
        PFC_Prod += Fonte_produit
    # print(t,numfours)
    PFC_Prod -= poche
    return numfours, PFC_Prod

def calcule_temps_poche_i(PFC_Consom, cadence, temps_traitement, Mg, Mgmin, eC):

    # Temps en minute admissible avant l'ajout dans le four de maintien
    # avant d'atteindre la fonte minimal (en Kg) dans le four de maintien
    fonte_four_consommable = PFC_Consom - PFCmin
    temps_epuis_fonte = fonte_four_consommable/cadence
    delai_avt_traitement_fonte_four = temps_epuis_fonte - temps_traitement

    # Temps en minute admissible avant l'ajout dans le four de maintien
    # avant d'atteindre le pourcentage minimal (%) dans le four de maintien
    Mgconsommable = Mg - Mgmin
    temps_epuis_mg = Mgconsommable/eC
    delai_avt_traitement_mg_four = temps_epuis_mg - temps_traitement


    # print(delai_avt_traitement_fonte_four, delai_avt_traitement_mg_four)
    # Temps en minute avant de lancer le traitement (Du four de fusion au four de maintien)
    delai_avt_traitement = min(delai_avt_traitement_fonte_four, delai_avt_traitement_mg_four)


    temps_poche_i = temps_traitement + delai_avt_traitement
    return np.ceil(temps_poche_i)

def ProdConsom(liste_masse_grappe, liste_quantite_mottes, nb_moules_heure, t_serie, temps_fusion, temps_chargement):
    """
    Calcule la consommation et la production de fonte.
    """
    # Initialisation des variables
    numfours = 0
    fonte_produit = 5000  # Fonte produite toutes les 45 min en kg
    PFC_Prodvalues = [0]  # Production de fonte en kg
    PFC_Consomvalues = [0]  # Consommation de fonte en kg
    times = [0]  # Temps en minutes

    t = 0
    PFC_Prod = PFC_Prodvalues[-1]
    PFC_Consom = PFC_Consomvalues[-1]

    
    
    

    # Mise à jour de la production de fonte
    t = temps_chargement + temps_fusion -1
    numfours, PFC_Prod = update_production(t, temps_chargement, temps_fusion, numfours, fonte_produit, PFC_Prod)
    PFC_Prodvalues.append(PFC_Prod)
    times.append(t)


    # Mise à jour de la production de fonte
    t = temps_chargement + temps_fusion 
    numfours, PFC_Prod = update_production(t, temps_chargement, temps_fusion, numfours, fonte_produit, PFC_Prod)
    PFC_Prodvalues.append(PFC_Prod)
    times.append(t)

    temps_poche_1, temps_poche_2, temps_poche_3 = 10, 10, 10 # temps traitement
    poche_1, poche_2, poche_3 = 1250, 1250, 1250

    
    t += temps_poche_1
    # Mise à jour de la consommation de fonte
    PFC_Consom += poche_1 
    PFC_Consomvalues.append(PFC_Consom)
    
    # Mise à jour de la production de fonte
    numfours, PFC_Prod = update_production(t, temps_chargement, temps_fusion, numfours, fonte_produit, PFC_Prod, poche_1)
    PFC_Prodvalues.append(PFC_Prod)
    times.append(t)


    t += temps_poche_2
    # Mise à jour de la consommation de fonte
    PFC_Consom += poche_2 
    PFC_Consomvalues.append(PFC_Consom)
    
    # Mise à jour de la production de fonte
    numfours, PFC_Prod = update_production(t, temps_chargement, temps_fusion, numfours, fonte_produit, PFC_Prod, poche_2)
    PFC_Prodvalues.append(PFC_Prod)
    times.append(t)


    t += temps_poche_3
    # Mise à jour de la consommation de fonte
    PFC_Consom += poche_3 
    PFC_Consomvalues.append(PFC_Consom)
    
    # Mise à jour de la production de fonte
    numfours, PFC_Prod = update_production(t, temps_chargement, temps_fusion, numfours, fonte_produit, PFC_Prod, poche_3)
    PFC_Prodvalues.append(PFC_Prod)
    times.append(t)

  


    Mgmin = 0.035 # en %
    Mgmax = 0.045 # en %
    eC = 0.0005
    temps_traitement = 10
    Mg = Mgmax

    Listetemps_poche_i = [0]


    t_avantpoche_i = t
    poche_i = 1250

    # Boucle principale consomation et production
    for masse_grappe, quantite_mottes in zip(liste_masse_grappe, liste_quantite_mottes):
        for _ in range(quantite_mottes):

            # Mise à jour de la consommation de fonte
            cadence = calculate_cadence_per_minute(nb_moules_heure, masse_grappe)
            PFC_Consom -= cadence  

            # Quand ajout poche_i
            if t == (Listetemps_poche_i[-1] + t_avantpoche_i) :
                temps_poche_i = calcule_temps_poche_i(PFC_Consom, cadence, temps_traitement, Mg, Mgmin, eC)
                Listetemps_poche_i.append(temps_poche_i)
                print(temps_poche_i)

                t_avantpoche_i = t
                # Mise à jour de la production de fonte
                numfours, PFC_Prod = update_production(t, temps_chargement, temps_fusion, numfours, fonte_produit, PFC_Prod,poche_i)
                Mg = Mgmax
                # Mise à jour de la production de fonte
                PFC_Consom += poche_i 

            temps_poche_i = 12

            # Mise à jour du temps
            t += 1

            # Mise à jour de la production de fonte
            if (t == (t_avantpoche_i + temps_poche_i)):
                pass
                # t_avantpoche_i = t
                # # Mise à jour de la production de fonte
                # numfours, PFC_Prod = update_production(t, temps_chargement, temps_fusion, numfours, fonte_produit, PFC_Prod,poche_i)
                # Mg = Mgmax
                # # Mise à jour de la production de fonte
                # PFC_Consom += poche_i 

            else :
                numfours, PFC_Prod = update_production(t, temps_chargement, temps_fusion, numfours, fonte_produit, PFC_Prod)
            
            times.append(t)
            PFC_Prodvalues.append(PFC_Prod)
            PFC_Consomvalues.append(PFC_Consom)



        t += t_serie
        times.append(t)
        PFC_Consomvalues.append(PFC_Consom)

        # Mise à jour de la production de fonte
        numfours, PFC_Prod = update_production(t, temps_chargement, temps_fusion, numfours, fonte_produit, PFC_Prod)
        PFC_Prodvalues.append(PFC_Prod)

    tmin = times[0]
    tmax = times[-1]
    return times, PFC_Consomvalues, PFC_Prodvalues, tmin, tmax


On a 4000 kg de fontes dans le four de Coulée avec un pourcentage de Mg = 0.0045 %.
On consonmme 36 Kg de fonte par minute dans le four de Coulée.
On perd 0.005 % de Mg par minute.

Tu es un mathématicien et tu dois Tracer la courbe d'évolution du Mg
dans le four de Coulée. 

Si tu as des questions ou s'il te manque certaines informations, n'hésite pas à me les demander !!

In [203]:
def calcule_temps_poche_i(PFC_Consom, cadence, temps_traitement, Mg, Mgmin, eC):

    # Temps en minute admissible avant l'ajout dans le four de maintien
    # avant d'atteindre la fonte minimal (en Kg) dans le four de maintien
    fonte_four_consommable = PFC_Consom - PFCmin
    temps_epuis_fonte = fonte_four_consommable/cadence
    delai_avt_traitement_fonte_four = temps_epuis_fonte - temps_traitement

    # Temps en minute admissible avant l'ajout dans le four de maintien
    # avant d'atteindre le pourcentage minimal (%) dans le four de maintien
    Mgconsommable = Mg - Mgmin
    temps_epuis_mg = Mgconsommable/eC
    delai_avt_traitement_mg_four = temps_epuis_mg - temps_traitement


    # print(delai_avt_traitement_fonte_four, delai_avt_traitement_mg_four)
    # Temps en minute avant de lancer le traitement (Du four de fusion au four de maintien)
    delai_avt_traitement = min(delai_avt_traitement_fonte_four, delai_avt_traitement_mg_four)


    temps_poche_i = temps_traitement + delai_avt_traitement
    return np.ceil(temps_poche_i)


PFCmin = 2500 # en kg
PFCmax = 3750 # en kg

Mgmin = 0.035 # en %
Mgmax = 0.045 # en %
eC = 0.0005 # en %/min
temps_traitement = 10  # en  min

Mg = Mgmax
masse_grappe = 35
PFC_Consom, cadence = PFCmax, nb_moules_heure / 60 * masse_grappe

calcule_temps_poche_i(PFC_Consom, cadence, temps_traitement, Mg, Mgmin, eC)

In [229]:
nb_fours = 6 # 8-9 fours pour HWS, 6 fours pour DiSA
Temps_fusion = 45 # en min
Temps_chargement = 30 # 30 - 45  min

# Les valeurs d'entrée Input 

nb_moules_heure = 160 
t_serie = 5 # 5 min

PFCmin = 2500 # en kg
PFCmax = 4000 # en kg

Mgmin = 0.035 # en %
Mgmax = 0.045 # en %
eC = 0.005 # en %
temps_traitement = 10  # en  min



ListeMasse_grappe = [35, 35, 35, 35] # en kg
ListeQuantite_mottes = [50, 50, 50, 50]

ListeMasse_grappe = [35] # en kg
ListeQuantite_mottes = [200]

times, PFC_Consomvalues, PFC_Prodvalues, tmin, tmax = ProdConsom(ListeMasse_grappe, ListeQuantite_mottes, nb_moules_heure, t_serie, Temps_fusion, Temps_chargement)

13.0
13.0
14.0
13.0
13.0
14.0
13.0
14.0
13.0
13.0
14.0
13.0
14.0
13.0
13.0


In [230]:
Plot_Prod(times, PFC_Prodvalues)
Plot_consom(times, PFC_Consomvalues, tmin, tmax, PFCmin, PFCmax)